# Step 2: Video Generation with Text Only

## Text-to-Video Generation with AWS Bedrock

### Introduction

Welcome to this comprehensive notebook on Text-to-Video Generation using AWS Bedrock's `nova-reel` model. This notebook demonstrates how to transform textual descriptions into dynamic video content using state-of-the-art AI technology.

Text-to-video generation represents a significant advancement in generative AI, enabling the creation of visual content directly from natural language descriptions. This capability opens up new possibilities for content creation, storytelling, and visual communication without requiring specialized video production skills or equipment.

In this notebook, we'll explore how AWS Bedrock's `nova-reel` model can interpret textual prompts and generate corresponding video sequences that match the described scenes, actions, and visual elements. The process involves several key steps:

* **Building Model Input** – Constructing the request payload for video generation
* **Starting Video Generation** – Initiating an asynchronous job using AWS Bedrock
* **Monitoring Job Status** – Tracking progress and waiting until completion
* **Downloading & Displaying Video** – Retrieving the generated video from S3 and rendering it in the notebook

This notebook builds upon the image processing capabilities demonstrated in the previous notebook, extending our pipeline to include video generation capabilities.

## Table of Contents

1. [Setup and Dependencies](#Setup-and-Dependencies)
2. [Configuration and Initialization](#Configuration-and-Initialization)
3. [Constructing the Model Input](#1.-Constructing-the-Model-Input)
4. [Initiating Video Generation](#2.-Initiating-Video-Generation)
5. [Monitoring Video Generation](#3.-Monitoring-Video-Generation)
6. [Downloading and Displaying the Video](#4.-Downloading-and-Displaying-the-Video)
7. [Running the Complete Workflow](#Running-the-Complete-Workflow)
8. [Results and Next Steps](#Results-and-Next-Steps)

***

## Setup and Dependencies

First, we'll import the necessary libraries and modules for video generation:

In [ ]:
import json
import time
import boto3
from pathlib import Path
from IPython.display import Video

## Configuration and Initialization

Next, we'll retrieve our stored variables from previous notebooks and initialize our AWS clients:

In [ ]:
%store -r BUCKET
%store -r OUTPUT_DIR

In [ ]:
bedrock_runtime = boto3.client("bedrock-runtime")
s3_client = boto3.client("s3")

We'll ensure our output directory exists for storing generated videos:

In [ ]:
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

## 1. Constructing the Model Input

### Function: `build_model_input`

This function builds a structured input payload for video generation.

#### Key Parameters

- `prompt`: The **text prompt** describing the desired scene.
- `image_base64`: Optional **reference image** in Base64 format.
- `duration`: The **length** of the video in seconds.
- `fps`: **Frame rate** of the video.
- `resolution`: **Output video resolution** (default: `1280x720`).
- `seed`: A **random seed** for consistent generation.

#### Functionality

- If an **image is provided**, it's added to the payload.
- Defines **video parameters** such as duration, FPS, and resolution.

<div class="alert alert-info">
<b>Note:</b> The seed parameter allows for reproducible results. Using the same seed with identical inputs will produce similar video outputs, which can be useful for iterative refinement.
</div>

In [ ]:
def build_model_input(prompt, image_base64=None, duration=6, fps=24, resolution="1280x720", seed=0):
    """
    Constructs the input payload for the Bedrock video generation model.

    Args:
        prompt (str): Text prompt for video generation.
        image_base64 (str): Base64 encoded image data.
        duration (int): Duration of the video in seconds.
        fps (int): Frames per second for the video.
        resolution (str): Resolution of the video.
        seed (int): Seed for deterministic generation.
    
    Returns:
        dict: The formatted model input payload.
    """
    output = {
        "taskType": "TEXT_VIDEO",
        "textToVideoParams": {
            "text": prompt,
            # "images": image_parameter
        },
        "videoGenerationConfig": {
            "durationSeconds": duration,
            "fps": fps,
            "dimension": resolution,
            "seed": seed
        },
    }
    if image_base64:
        output['textToVideoParams']['images'] = [{"format": "png", "source": {"bytes": image_base64}}]
    return output

## 2. Initiating Video Generation

### Function: `start_video_generation`

This function submits a **video generation job** to AWS Bedrock.

#### Key Parameters

- `bedrock_runtime`: AWS **Bedrock runtime client**.
- `bucket`: The **S3 bucket** where output videos will be stored.
- `model_input`: The formatted **JSON payload** for video generation.

#### Process

1. Calls `start_async_invoke()` to trigger video creation.
2. Saves results in the **specified S3 bucket**.
3. Returns an **invocation ARN** to track progress.

<div class="alert alert-warning">
<b>Important:</b> Video generation is an asynchronous process that may take several minutes to complete. The function returns immediately with an ARN that can be used to monitor progress.
</div>

In [ ]:
def start_video_generation(bedrock_runtime, bucket, model_input):
    """
    Starts a video generation job using Bedrock's Text-to-Video model.

    Args:
        bedrock_runtime: Initialized Bedrock Runtime client.
        bucket (str): S3 bucket for storing output video.
        prompt (str): Text prompt for video generation.

    Returns:
        dict: Invocation response from the Bedrock API.
    """

    try:
        # Start the asynchronous video generation job
        invocation = bedrock_runtime.start_async_invoke(
            modelId="amazon.nova-reel-v1:0",
            modelInput=model_input,
            outputDataConfig={
                "s3OutputDataConfig": {"s3Uri": f"s3://{bucket}"}
            }
        )
        print("Video generation job started successfully.")
        return invocation
    except Exception as e:
        error_message = e.response["Error"]["Message"] if "Error" in e.response else str(e)
        print(f"Error starting video generation: {error_message}")
        raise

## 3. Monitoring Video Generation

### Function: `monitor_video_generation`

This function continuously checks the **status** of the video generation job.

#### Key Parameters

- `bedrock_runtime`: AWS **Bedrock runtime client**.
- `invocation_arn`: The **job ID** returned from `start_video_generation`.

#### Process

1. Polls AWS Bedrock for **job status** (`InProgress`, `Completed`, or `Failed`).
2. If **successful**, retrieves the **S3 URI** of the video.
3. If **failed**, logs the error and stops execution.

<div class="alert alert-info">
<b>Note:</b> The function checks the job status every 60 seconds to avoid excessive API calls while still providing timely updates.
</div>

In [ ]:
def monitor_video_generation(bedrock_runtime, invocation_arn):
    """
    Monitors the status of the video generation job until completion.

    Args:
        bedrock_runtime: Initialized Bedrock Runtime client.
        invocation_arn (str): ARN of the video generation invocation.

    Returns:
        str: S3 URI of the generated video.
    """
    status = "InProgress"
    while status not in ["Completed", "Failed"]:
        response = bedrock_runtime.get_async_invoke(invocationArn=invocation_arn)
        status = response["status"]

        if status == "Completed":
            s3_uri = response["outputDataConfig"]["s3OutputDataConfig"]["s3Uri"]
            print(f"Video generation completed. Video available at: {s3_uri}/output.mp4")
            return f"{s3_uri}/output.mp4"
        elif status == "InProgress":
            print(f"Job {invocation_arn} is in progress. Started at: {response['submitTime']}")
        elif status == "Failed":
            print(f"Job {invocation_arn} failed. Reason: {response['failureMessage']}")
            raise RuntimeError("Video generation failed.")
        time.sleep(60)

## 4. Downloading and Displaying the Video

### Function: `download_and_display_video`

This function fetches the **generated video from S3** and plays it in the notebook.

#### Key Parameters

- `s3_client`: AWS **S3 client** for file operations.
- `bucket`: The **S3 bucket** storing the video.
- `video_uri`: The **S3 path** to the generated video.
- `output_dir`: **Local directory** for saving the video.

#### Process

1. Extracts the **video filename** from the S3 URI.
2. Downloads the video **from S3 to local storage**.
3. Uses **IPython's `Video` widget** to display it inline.

In [ ]:
def download_and_display_video(s3_client, bucket, video_uri, output_dir):
    """
    Downloads the generated video from S3 and displays it in the notebook.

    Args:
        s3_client: Initialized S3 client.
        bucket (str): S3 bucket containing the video.
        video_uri (str): URI of the video in S3.

    Returns:
        Video: IPython Video object for display.
    """
    local_path = f"{output_dir}/{video_uri.split('/')[3]}.mp4"
    print(local_path)
    s3_client.download_file(bucket, "/".join(video_uri.split("/")[3:]), local_path)
    print(f"Video downloaded to {local_path}.")
    return Video(local_path)

## Running the Complete Workflow

Now we'll put everything together to generate a video from a text prompt. First, let's define our prompt:

In [ ]:
PROMPT = "Closeup of a large seashell in the sand, gentle waves flow around the shell. Camera zoom in."

Next, we'll build the model input payload using our prompt:

In [ ]:
# Build the model input payload
model_input = build_model_input(PROMPT)
model_input

<div class="alert alert-warning">
<b>Important:</b> The following cell will start the video generation process, which may take several minutes to complete. The notebook will poll for status updates every 60 seconds.
</div>

In [ ]:
# Main workflow
try:
    # Step 1: Start video generation
    invocation_response = start_video_generation(bedrock_runtime, BUCKET, model_input)
    invocation_arn = invocation_response["invocationArn"]

    # Step 2: Monitor the job
    video_s3_uri = monitor_video_generation(bedrock_runtime, invocation_arn)

    # Step 3: Download and display the video
    local_path = download_and_display_video(s3_client, BUCKET, video_s3_uri, OUTPUT_DIR)
except Exception as e:
    print(f"An error occurred during video generation: {str(e)}")

If you need to display the video again, you can use the following cell:

In [ ]:
download_and_display_video(s3_client, BUCKET, video_s3_uri, OUTPUT_DIR)

## Results and Next Steps

<div class="alert alert-success">
<b>🎉 Congratulations!</b> You have successfully completed the text-to-video generation notebook!

Key accomplishments:
- ✅ Created a structured input payload for video generation
- ✅ Initiated an asynchronous video generation job
- ✅ Monitored the job status until completion
- ✅ Downloaded and displayed the generated video
</div>

### Next Steps

In the next notebook, we'll explore how to enhance video generation by incorporating both text prompts and reference images, which can provide more control over the visual style and content of the generated videos.

Proceed to [Step 3: Video Generation with Text and Image](_03_video_gen_text_image.ipynb) to learn how to use image conditioning for video generation.